## Statsmodel Forecast with Wallaroo Features: Data Connection

Wallaroo Connections are definitions set by MLOps engineers that are used by other Wallaroo users for connection information to a data source.

This provides MLOps engineers a method of creating and updating connection information for data stores:  databases, Kafka topics, etc.  Wallaroo Connections are composed of three main parts:

* Name:  The unique name of the connection.
* Type:  A user defined string that designates the type of connection.  This is used to organize connections.
* Details:  Details are a JSON object containing the information needed to make the connection.  This can include data sources, authentication tokens, etc.

Wallaroo Connections are only used to store the connection information used by other processes to create and use external connections.  The user still has to provide the libraries and other elements to actually make and use the conneciton.

The primary advantage is Wallaroo connections allow scripts and other code to retrieve the connection details directly from their Wallaroo instance, then refer to those connection details.  They don't need to know what those details actually - they can refer to them in their code to make their code more flexible.

For this step, we will use a Google BigQuery dataset to retrieve the inference information, predict the next month of sales, then store those predictions into another table.  This will use the Wallaroo Connection feature to create a Connection, assign it to our workspace, then perform our inferences by using the Connection details to connect to the BigQuery dataset and tables.

## Prerequisites

* A Wallaroo instance version 2023.2.1 or greater.
* [Google Authentication Credentials](https://cloud.google.com/docs/authentication/external/set-up-adc).  This tutorial allows any authenticated Google account to view the data in the reference dataset and tables.

## References

* [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Python Models](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-python/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)
* [Wallaroo SDK Essentials Guide: Data Connections Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnections/)

## Statsmodel Forecast Connection Steps

### Import Libraries

The first step is to import the libraries that we will need.

In [1]:
import json
import os
import datetime

import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
import numpy as np

from resources import simdb
from resources import util

pd.set_option('display.max_colwidth', None)

# for Big Query connections
from google.cloud import bigquery
from google.oauth2 import service_account
import db_dtypes

import time

In [2]:
display(wallaroo.__version__)

'2023.2.1'

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [3]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

wallarooPrefix = "doc-test."
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Set Configurations

The following will set the workspace, model name, and pipeline that will be used for this example.  If the workspace or pipeline already exist, then they will assigned for use in this example.  If they do not exist, they will be created based on the names listed below.

Workspace names must be unique.  To allow this tutorial to run in the same Wallaroo instance for multiple users, set the `suffix` variable or share the workspace with other users.

#### Set Configurations References

* [Wallaroo SDK Essentials Guide: Workspace Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-workspace/)
* [Wallaroo SDK Essentials Guide: Pipeline Management](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/)

In [4]:
# used for unique connection names

import string
import random

suffix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))

suffix='jchnew'

workspace_name = f'forecast-model-workshop{suffix}'

pipeline_name = 'forecast-workshop-pipeline'

### Set the Workspace and Pipeline

The workspace will be either used or created if it does not exist, along with the pipeline.  The models uploaded in the Upload and Deploy tutorial are referenced in this step.

In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

pipeline = get_pipeline(pipeline_name)

# Get the most recent version of a model in the workspace
# Assumes that the most recent version is the first in the list of versions.
# wl.get_current_workspace().models() returns a list of models in the current workspace

def get_model(mname):
    modellist = wl.get_current_workspace().models()
    model = [m.versions()[0] for m in modellist if m.name() == mname]
    if len(model) <= 0:
        raise KeyError(f"model {mname} not found in this workspace")
    return model[0]

# upload three models:  the control and two challengers

control_model_name = 'forecast-control-model'
challenger01_model_name = 'forecast-challenger01-model'
challenger02_model_name = 'forecast-challenger02-model'

# retrieve the models

bike_day_model = get_model(control_model_name)

challenger_model_01 = get_model(challenger01_model_name)

challenger_model_02 = get_model(challenger02_model_name)


### Deploy the Pipeline

We will now add the uploaded model as a step for the pipeline, then deploy it.  The pipeline configuration will allow for multiple replicas of the pipeline to be deployed and spooled up in the cluster.  Each pipeline replica will use 0.25 cpu and 512 Gi RAM.

In [6]:
# Set the deployment to allow for additional engines to run
# Undeploy and clear the pipeline in case it was used in other demonstrations
pipeline.undeploy()
pipeline.clear()
deploy_config = (wallaroo.DeploymentConfigBuilder()
                        .replica_count(1)
                        .replica_autoscale_min_max(minimum=2, maximum=5)
                        .cpus(0.25)
                        .memory("512Mi")
                        .build()
                    )

pipeline.add_model_step(bike_day_model)
# pipeline.add_model_step(step)
pipeline.deploy(deployment_config = deploy_config)

name,forecast-workshop-pipeline
created,2023-07-28 15:14:53.116222+00:00
last_updated,2023-07-28 15:22:36.379347+00:00
deployed,True
tags,
versions,"47db2792-94dd-4e71-b3ad-a9ae06b2cf55, 721b885f-46c9-4009-8816-1237f5be4616, 11a7a862-81b2-459e-a59a-ee1d2394d83b, 324f1882-dd5b-4052-bbab-e77a48ab8f47, 45942248-a46c-417e-91d1-20fac087f2a4, e86d795d-71b0-4537-85a3-261806fc0c29, 5efbf5f1-74c7-46f0-9f66-9866f66ead89"
steps,forecast-control-model


### Create the Connection

For this demonstration, the connection set to a specific file on a GitHub repository.  The connection details can be anything that can be stored in JSON:  connection URLs, tokens, etc.

This connection will set a URL to pull a file from GitHub, then use the file contents to perform an inference.

Wallaroo connections are created through the Wallaroo Client `create_connection(name, type, details)` method.  See the [Wallaroo SDK Essentials Guide: Data Connections Management guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-dataconnections/) for full details.

Wallaroo connections are retrieved with the Wallaroo Client `get_connection`

Note that connection names must be unique across the Wallaroo instance.  The sample code below assumes that the connection with the same name was either previously created with the proper credentials, or will be created through this step.

In [7]:
# set the connection information for other steps

forecast_connection_input_name = f'statsmodel-bike-rentals-{suffix}'
forecast_connection_input_type = "HTTP"
forecast_connection_input_argument = { 
    "url": "https://raw.githubusercontent.com/WallarooLabs/Workshops/main/forecast/forecast-retail/data/testdata_dict.json"
    }

# if the connection with the same name exists, use it.  Otherwise, create it.
def get_connection(name, type, details):
    connection = None
    for cn in wl.list_connections():
        if cn.name() == name:
            connection= cn
    if(connection == None):
        connection = wl.create_connection(name=name, 
                                         connection_type=type, 
                                         details=details)
    return connection


statsmodel_connection = get_connection(forecast_connection_input_name,
                                       forecast_connection_input_type,
                                       forecast_connection_input_argument)

### Add Connection to Workspace

We'll now add the connection to our workspace so it can be retrieved by other workspace users.  The method Workspace `add_connection(connection_name)` adds a Data Connection to a workspace.

In [8]:
workspace.add_connection(forecast_connection_input_name)
workspace.list_connections()

name,connection type,details,created at,linked workspaces
statsmodel-bike-rentals-jchnew,HTTP,*****,2023-07-28T15:23:20.069629+00:00,['forecast-model-workshopjchnew']


### Retrieve Connection from Workspace

To simulate a data scientist's procedural flow, we'll now retrieve the connection from the workspace.

The method Workspace `list_connections()` displays a list of connections attached to the workspace. By default the details field is obfuscated.  Specific connections are retrieved by specifying their position in the returned list.

In [9]:
forecast_connection = workspace.list_connections()[0]
display(forecast_connection)

Field,Value
Name,statsmodel-bike-rentals-jchnew
Connection Type,HTTP
Details,*****
Created At,2023-07-28T15:23:20.069629+00:00
Linked Workspaces,['forecast-model-workshopjchnew']


### Run Inference with Connection

We'll now retrieve sample data through the Wallaroo connection, and perform a sample inference.  The connection details are retrieved through the Connection `details()` method.

In [10]:
import requests

response = requests.get(
                    forecast_connection.details()['url']
                )

inferencedata = response.json()

results = pipeline.infer(inferencedata)

display(results)



[{'forecast': [1764, 1749, 1743, 1741, 1740, 1740, 1740]}]

### Undeploy the Pipeline

Undeploy the pipeline and return the resources back to the Wallaroo instance.

In [11]:
pipeline.undeploy()

name,forecast-workshop-pipeline
created,2023-07-28 15:14:53.116222+00:00
last_updated,2023-07-28 15:22:36.379347+00:00
deployed,False
tags,
versions,"47db2792-94dd-4e71-b3ad-a9ae06b2cf55, 721b885f-46c9-4009-8816-1237f5be4616, 11a7a862-81b2-459e-a59a-ee1d2394d83b, 324f1882-dd5b-4052-bbab-e77a48ab8f47, 45942248-a46c-417e-91d1-20fac087f2a4, e86d795d-71b0-4537-85a3-261806fc0c29, 5efbf5f1-74c7-46f0-9f66-9866f66ead89"
steps,forecast-control-model
